# VGG16 Transfer Learning
The source is from [agoila/transfer-learning](https://github.com/agoila/transfer-learning), I also implement it locally [Transfer_Learning.ipynb](../transfer-learning/Transfer_Learning.ipynb)

Now, I will redo it but using the VGG16 model from [VGG in TensorFlow](https://www.cs.toronto.edu/~frossard/post/vgg16/)

In [1]:
%matplotlib inline
import skimage
import skimage.io
import skimage.transform
import tensorflow as tf
import numpy as np
import os
from matplotlib import pyplot as plt
from vgg16_transfer_learn import vgg16

In [2]:
data_dir = '../transfer-learning/flower_photos/'
contents = os.listdir(data_dir)
classes = [each for each in contents if os.path.isdir(data_dir + each)]
print(classes)

['daisy', 'sunflowers', 'roses', 'dandelion', 'tulips']


In [3]:
# returns image of shape [224, 224, 3]
# [height, width, depth]
def load_image(path):
    # load image
    img = skimage.io.imread(path)
    img = img / 255.0
    assert (0 <= img).all() and (img <= 1.0).all()
    # print("Original Image Shape:", img.shape)
    # we crop image from center
    short_edge = min(img.shape[:2])
    # print("short_edge:", short_edge)
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    # print(yy, xx)
    crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
    # resize to 224, 224
    resized_img = skimage.transform.resize(crop_img, (224, 224), mode='constant')
    return resized_img

In [4]:
# class_path = data_dir + classes[0]
# files = os.listdir(class_path)
# img = load_image(os.path.join(class_path, files[0]))

In [5]:
# Set the batch size higher if you can fit in in your GPU memory
batch_size = 10
codes_list = []
labels = []
batch = []

codes = None

with tf.Session() as sess:
    
    # TODO: Build the vgg network here
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    vgg = vgg16(input_, 'vgg16_weights.npz', sess)

    for each in classes:
        print("Starting {} images".format(each))
        class_path = data_dir + each
        files = os.listdir(class_path)
        for ii, file in enumerate(files, 1):
            # Add images to the current batch
            # utils.load_image crops the input images for us, from the center
            img = load_image(os.path.join(class_path, file))
            batch.append(img.reshape((1, 224, 224, 3)))
            labels.append(each)
            
            # Running the batch through the network to get the codes
            if ii % batch_size == 0 or ii == len(files):
                
                # Image batch to pass to VGG network
                images = np.concatenate(batch)
                
                # TODO: Get the values from the relu6 layer of the VGG network
                feed_dict = {input_: images}
                codes_batch = sess.run(vgg.fc1, feed_dict=feed_dict)
                
                # Here I'm building an array of the codes
                if codes is None:
                    codes = codes_batch
                else:
                    codes = np.concatenate((codes, codes_batch))
                
                # Reset to start building the next batch
                batch = []
                print('{} images processed'.format(ii))

build model started
0 conv1_1_W (3, 3, 3, 64)
1 conv1_1_b (64,)
2 conv1_2_W (3, 3, 64, 64)
3 conv1_2_b (64,)
4 conv2_1_W (3, 3, 64, 128)
5 conv2_1_b (128,)
6 conv2_2_W (3, 3, 128, 128)
7 conv2_2_b (128,)
8 conv3_1_W (3, 3, 128, 256)
9 conv3_1_b (256,)
10 conv3_2_W (3, 3, 256, 256)
11 conv3_2_b (256,)
12 conv3_3_W (3, 3, 256, 256)
13 conv3_3_b (256,)
14 conv4_1_W (3, 3, 256, 512)
15 conv4_1_b (512,)
16 conv4_2_W (3, 3, 512, 512)
17 conv4_2_b (512,)
18 conv4_3_W (3, 3, 512, 512)
19 conv4_3_b (512,)
20 conv5_1_W (3, 3, 512, 512)
21 conv5_1_b (512,)
22 conv5_2_W (3, 3, 512, 512)
23 conv5_2_b (512,)
24 conv5_3_W (3, 3, 512, 512)
25 conv5_3_b (512,)
26 fc6_W (25088, 4096)
27 fc6_b (4096,)
28 fc7_W (4096, 4096)
29 fc7_b (4096,)
30 fc8_W (4096, 1000)
31 fc8_b (1000,)
build model finished: 11s
Starting daisy images


/home/ryanyao/anaconda3/envs/tensorflow/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


10 images processed
20 images processed
30 images processed
40 images processed
50 images processed
60 images processed
70 images processed
80 images processed
90 images processed
100 images processed
110 images processed
120 images processed
130 images processed
140 images processed
150 images processed
160 images processed
170 images processed
180 images processed
190 images processed
200 images processed
210 images processed
220 images processed
230 images processed
240 images processed
250 images processed
260 images processed
270 images processed
280 images processed
290 images processed
300 images processed
310 images processed
320 images processed
330 images processed
340 images processed
350 images processed
360 images processed
370 images processed
380 images processed
390 images processed
400 images processed
410 images processed
420 images processed
430 images processed
440 images processed
450 images processed
460 images processed
470 images processed
480 images processed
4

In [6]:
# write codes to file
with open('codes', 'w') as f:
    codes.tofile(f)
    
# write labels to file
import csv
with open('labels', 'w') as f:
    writer = csv.writer(f, delimiter='\n')
    writer.writerow(labels)

# Building the Classifier
Now that we have codes for all the images, we can build a simple classifier on top of them. The codes behave just like normal input into a simple neural network. Below I'm going to have you do most of the work.

In [7]:
# read codes and labels from file
import csv

with open('labels') as f:
    reader = csv.reader(f, delimiter='\n')
    labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('codes') as f:
    codes = np.fromfile(f, dtype=np.float32)
    codes = codes.reshape((len(labels), -1))

In [8]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
labels_vecs = lb.fit_transform(labels) # Your one-hot encoded labels array here

In [9]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2)

train_idx, val_idx = next(sss.split(codes, labels_vecs))
validSplitSize = int(len(val_idx)/2)

val_idx, test_idx = val_idx[:validSplitSize], val_idx[validSplitSize:]

train_x, train_y = codes[train_idx], labels_vecs[train_idx]
val_x, val_y = codes[val_idx], labels_vecs[val_idx]
test_x, test_y = codes[test_idx], labels_vecs[test_idx]

In [10]:
print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)
print("Test shapes (x, y):", test_x.shape, test_y.shape)

Train shapes (x, y): (2936, 4096) (2936, 5)
Validation shapes (x, y): (367, 4096) (367, 5)
Test shapes (x, y): (367, 4096) (367, 5)


In [11]:
inputs_ = tf.placeholder(tf.float32, shape=[None, codes.shape[1]])
labels_ = tf.placeholder(tf.int64, shape=[None, labels_vecs.shape[1]])

# TODO: Classifier layers and operations
fc = tf.contrib.layers.fully_connected(inputs_, 512)

logits = tf.contrib.layers.fully_connected(fc, labels_vecs.shape[1], activation_fn=None)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels_, logits=logits)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer().minimize(cost)

# Operations for validation/test accuracy
predicted = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
def get_batches(x, y, n_batches=10):
    """ Return a generator that yields batches from arrays x and y. """
    batch_size = len(x)//n_batches
    
    for ii in range(0, n_batches*batch_size, batch_size):
        # If we're not on the last batch, grab data with size batch_size
        if ii != (n_batches-1)*batch_size:
            X, Y = x[ii: ii+batch_size], y[ii: ii+batch_size] 
        # On the last batch, grab the rest of the data
        else:
            X, Y = x[ii:], y[ii:]
        # I love generators
        yield X, Y

In [13]:
epochs = 10
iteration = 0
saver = tf.train.Saver()
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    # TODO: Your training code here
    for e in range(epochs):
        for x, y in get_batches(train_x, train_y):
            loss, _ = sess.run([cost, optimizer], feed_dict={inputs_: x, labels_: y})
            print("Epoch: {}/{}".format(e+1, epochs),
                  "Iteration: {}".format(iteration),
                  "Training Loss: {:.5f}".format(loss))
            iteration += 1
            
            if iteration % 5 == 0:
                valid_acc = sess.run(accuracy, feed_dict={inputs_: val_x, labels_: val_y})
                print("Epoch: {}/{}".format(e+1, epochs),
                      "Iteration: {}".format(iteration),
                      "Validation accuracy: {:.5f}".format(valid_acc))
        
    saver.save(sess, "checkpoints/flowers.ckpt")

Epoch: 1/10 Iteration: 0 Training Loss: 8.13290
Epoch: 1/10 Iteration: 1 Training Loss: 27.75088
Epoch: 1/10 Iteration: 2 Training Loss: 30.47755
Epoch: 1/10 Iteration: 3 Training Loss: 10.46250
Epoch: 1/10 Iteration: 4 Training Loss: 4.56607
Epoch: 1/10 Iteration: 5 Validation accuracy: 0.62943
Epoch: 1/10 Iteration: 5 Training Loss: 5.14031
Epoch: 1/10 Iteration: 6 Training Loss: 6.91324
Epoch: 1/10 Iteration: 7 Training Loss: 7.15179
Epoch: 1/10 Iteration: 8 Training Loss: 6.94140
Epoch: 1/10 Iteration: 9 Training Loss: 5.46307
Epoch: 1/10 Iteration: 10 Validation accuracy: 0.71117
Epoch: 2/10 Iteration: 10 Training Loss: 4.29187
Epoch: 2/10 Iteration: 11 Training Loss: 2.07225
Epoch: 2/10 Iteration: 12 Training Loss: 1.65150
Epoch: 2/10 Iteration: 13 Training Loss: 1.20557
Epoch: 2/10 Iteration: 14 Training Loss: 0.95366
Epoch: 2/10 Iteration: 15 Validation accuracy: 0.77929
Epoch: 2/10 Iteration: 15 Training Loss: 1.17806
Epoch: 2/10 Iteration: 16 Training Loss: 1.18930
Epoch: 2/1

# Testing
Below you see the test accuracy. You can also see the predictions returned for images.

In [14]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    feed = {inputs_: test_x,
            labels_: test_y}
    test_acc = sess.run(accuracy, feed_dict=feed)
    print("Test accuracy: {:.4f}".format(test_acc))

INFO:tensorflow:Restoring parameters from checkpoints/flowers.ckpt
Test accuracy: 0.8774
